<a href="https://www.kaggle.com/code/daniyalatta/ps-s5-09-lightgbm-a-z-best-for-dataset-understand?scriptVersionId=260425120" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="text-align:center; border-radius:25px 70px; padding:15px; color:#ffffff; margin:0; font-size:180%; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden">
  <b>
    <h3  style="font-weight:150;color:#ffffff; font-size:120%">🚀 Your upvote can motivate me to share more useful notebooks!</h3>
    <h4 style=" font-weight:120; color:#ffffff; font-size:100%">Rate My Notebook Design</h4>
  </b>
</div>


<!-- ================= HEADER ================= -->
<div id="top" style="text-align:center; border-radius:25px 70px; padding:14px 18px; color:#ffffff; margin:0 0 18px 0; font-size:200%; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden; letter-spacing:.3px;">
  <b>LightGBM Regression Pipeline (Kaggle Playground S5E9)</b>
</div>

<!-- ================= TOC ================= -->
<div style="font-family:Inter, system-ui, -apple-system, 'Segoe UI', Roboto, Arial, sans-serif; background:#24a695; color:#ffffff; padding:14px 16px; border-radius:16px; border:1px solid #1d7e6f; margin-bottom:20px;">
  <div style="font-weight:700; font-size:18px; margin-bottom:8px;"><h1 style="margin:0; color:#ffffff;">Table of contents</h1></div>
  <ol style="margin:0; font-size:16px; padding-left:22px; line-height:1.7;">
    <li><a href="#Dependencies" style="color:#ffffff; text-decoration:none;">Dependencies</a></li>
    <li><a href="#Utility-Functions" style="color:#ffffff; text-decoration:none;">Utility Functions</a></li>
    <li><a href="#Data-Loading" style="color:#ffffff; text-decoration:none;">Data Loading</a></li>
    <li><a href="#Data-Preprocessing" style="color:#ffffff; text-decoration:none;">Data Preprocessing</a></li>
    <li><a href="#Model-Training-and-Prediction" style="color:#ffffff; text-decoration:none;">Model Training and Prediction</a></li>
    <li><a href="#Save-Outputs" style="color:#ffffff; text-decoration:none;">Save Outputs</a></li>
    <li><a href="#Evaluate-Performance" style="color:#ffffff; text-decoration:none;">Evaluate Performance</a></li>
  </ol>
</div>


<!-- ================= HEADER + BADGES MERGED ================= -->
<div style="border-radius:25px 70px; padding:14px 18px; margin:0 0 18px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">
    <b><h2 style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">Dependencies</h2></b>
  </div>
</div>

<div style="display:flex; gap:8px; flex-wrap:wrap; margin-top:10px; font-family:Inter, system-ui, 'Segoe UI', Roboto, Arial, sans-serif;">
  <span style="background:#e2e8f0; color:#0f172a; padding:6px 10px; border-radius:999px; font-size:12px; border:1px solid #cbd5e1;">pandas</span>
  <span style="background:#dbeafe; color:#1e3a8a; padding:6px 10px; border-radius:999px; font-size:12px; border:1px solid #bfdbfe;">numpy</span>
  <span style="background:#dcfce7; color:#14532d; padding:6px 10px; border-radius:999px; font-size:12px; border:1px solid #bbf7d0;">lightgbm</span>
  <span style="background:#e2e8f0; color:#0f172a; padding:6px 10px; border-radius:999px; font-size:12px; border:1px solid #cbd5e1;">scikit-learn</span>
  <span style="background:#dbeafe; color:#1e3a8a; padding:6px 10px; border-radius:999px; font-size:12px; border:1px solid #bfdbfe;">gc</span>
  <span style="background:#dcfce7; color:#14532d; padding:6px 10px; border-radius:999px; font-size:12px; border:1px solid #bbf7d0;">warnings</span>
</div>


In [1]:
# Dependencies
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
import gc
import warnings
warnings.filterwarnings("ignore")

<!-- ================= HEADER ================= -->
<div style="border-radius:25px 70px; padding:14px 18px; margin:0 0 18px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Utility Functions</h3></b>
  </div>
</div>

<!-- ================= BADGES ================= -->
<div style="display:flex; gap:8px; flex-wrap:wrap; margin-top:10px; font-family:Inter, system-ui, 'Segoe UI', Roboto, Arial, sans-serif;">
  <span style="background:#dbeafe; color:#1e3a8a; padding:6px 12px; border-radius:999px; font-size:13px; border:1px solid #bfdbfe;">
    Free Memory — Clears memory to optimize performance
  </span>
</div>


In [2]:
# Free Memory
def free():
    gc.collect()

<!-- ================= HEADER ================= -->
<div style="border-radius:25px 70px; padding:14px 18px; margin:0 0 18px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Calculate Mode</h3></b>
  </div>
</div>

<!-- ================= BADGE ================= -->
<div style="display:flex; gap:8px; flex-wrap:wrap; margin-top:10px; font-family:Inter, system-ui, 'Segoe UI', Roboto, Arial, sans-serif;">
  <span style="background:#dcfce7; color:#14532d; padding:6px 12px; border-radius:999px; font-size:13px; border:1px solid #bbf7d0;">
    Computes the mode of a series, handling missing values
  </span>
</div>


In [3]:
# Calculate Mode
def calc_mode(x):
    x = x.dropna()
    if len(x) == 0:
        return np.nan
    return pd.Series(x).mode()[0]

### RMSE Metric
Calculates the Root Mean Squared Error (RMSE) for evaluation.

In [4]:
# RMSE Metric
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

<!-- ================= HEADER ================= -->
<div style="border-radius:25px 70px; padding:14px 18px; margin:0 0 18px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">
    <b><h2 style="margin:0; color:#ffffff;">Data Loading</h2></b>
  </div>
</div>

<!-- ================= BADGE ================= -->
<div style="display:flex; gap:8px; flex-wrap:wrap; margin-top:10px; font-family:Inter, system-ui, 'Segoe UI', Roboto, Arial, sans-serif;">
  <span style="background:#dbeafe; color:#1e3a8a; padding:6px 12px; border-radius:999px; font-size:13px; border:1px solid #bfdbfe;">
    Loads the training, test, and sample submission files from the Kaggle dataset
  </span>
</div>


In [5]:
# Data Loading
PATH = "/kaggle/input/playground-series-s5e9/"
dt = pd.read_csv(f"{PATH}train.csv")
dtest = pd.read_csv(f"{PATH}test.csv")
sub = pd.read_csv(f"{PATH}sample_submission.csv")
sub_temp = sub.iloc[:, 1].copy()
# Debug: Inspect raw data
print("Raw train data shape:", dt.shape)
print("Raw train data non-NaN counts:", dt.notna().sum())
print("Raw train data types:", dt.dtypes)

Raw train data shape: (524164, 11)
Raw train data non-NaN counts: id                           524164
RhythmScore                  524164
AudioLoudness                524164
VocalContent                 524164
AcousticQuality              524164
InstrumentalScore            524164
LivePerformanceLikelihood    524164
MoodScore                    524164
TrackDurationMs              524164
Energy                       524164
BeatsPerMinute               524164
dtype: int64
Raw train data types: id                             int64
RhythmScore                  float64
AudioLoudness                float64
VocalContent                 float64
AcousticQuality              float64
InstrumentalScore            float64
LivePerformanceLikelihood    float64
MoodScore                    float64
TrackDurationMs              float64
Energy                       float64
BeatsPerMinute               float64
dtype: object


<!-- ================= H2 HEADER ================= -->
<div style="border-radius:25px 70px; padding:14px 18px; margin:0 0 18px 0;
            font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">
    <b><h2 style="margin:0; color:#ffffff;">Data Preprocessing</h2></b>
  </div>
</div>

<!-- ================= BADGES OVERVIEW ================= -->
<div style="display:flex; gap:10px; flex-wrap:wrap; justify-content:center;
            margin:12px 0; font-family:Inter, system-ui, 'Segoe UI', Roboto, Arial, sans-serif;">
  <span style="background:#dbeafe; color:#1e3a8a; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bfdbfe;">Identify Target</span>
  <span style="background:#dcfce7; color:#14532d; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bbf7d0;">Merge Train and Test</span>
  <span style="background:#fef9c3; color:#854d0e; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fde68a;">Remove Duplicated Columns</span>
  <span style="background:#e9d5ff; color:#581c87; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #d8b4fe;">Remove High-Null Columns</span>
  <span style="background:#fcd5ce; color:#881337; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fda4af;">Remove Single-Value Columns</span>
  <span style="background:#d1fae5; color:#065f46; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #a7f3d0;">Convert potential numeric columns to numeric type</span>
  <span style="background:#fde68a; color:#92400e; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fcd34d;">Encode Categorical Columns</span>
  <span style="background:#c7d2fe; color:#3730a3; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #a5b4fc;">Remove Low-Variance Features</span>
  <span style="background:#fcd5ce; color:#9d174d; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fca5a5;">Frequency Encoding</span>
  <span style="background:#d1fae5; color:#064e3b; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #86efac;">Handle Missing Values</span>
  <span style="background:#fde68a; color:#78350f; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fcd34d;">Split Train and Test</span>
  <span style="background:#c7d2fe; color:#1e3a8a; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #a5b4fc;">Preprocessing Pipeline</span>
</div>

<!-- Badge 1 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#bfdbfe; color:#1e3a8a; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bfdbfe;">
      Identify Target
    </span>
  </div>
</div>

In [6]:
# Identify Target
target = [col for col in dt.columns if col not in dtest.columns]
Y = dt[target].max(axis=1)
dt = dt.drop(columns=target)

<!-- Badge 2 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#dcfce7; color:#14532d; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bbf7d0;">
      Merge Train and Test
    </span>
  </div>
</div>

In [7]:
# Merge Train and Test
dt['fuente'] = 'train'
dtest['fuente'] = 'test'
dt_total = pd.concat([dt, dtest], ignore_index=True)

<!-- Badge 3 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#fef9c3; color:#854d0e; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fde68a;">
      Remove Duplicated Columns
    </span>
  </div>
</div>

In [8]:
# Remove Duplicated Columns
cols_duplicated = dt_total.columns[dt_total.T.duplicated()].tolist()
if cols_duplicated:
    dt_total = dt_total.drop(columns=cols_duplicated)
    print(f"Removed duplicated columns: {cols_duplicated}")

<!-- Badge 4 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#e9d5ff; color:#581c87; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #d8b4fe;">
      Remove High-Null Columns
    </span>
  </div>
</div>

In [9]:
# Remove High-Null Columns
null_cols = dt_total.columns[dt_total.isnull().mean() >= 0.95].tolist()
if null_cols:
    dt_total = dt_total.drop(columns=null_cols)
    print(f"Removed columns with >= 95% nulls: {null_cols}")

<!-- Badge 5 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#fcd5ce; color:#881337; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fda4af;">
      Remove Single-Value Columns
    </span>
  </div>
</div>

In [10]:
# Remove Single-Value Columns
n_unique = dt_total.nunique()
one_value_cols = n_unique[n_unique == 1].index.tolist()
if one_value_cols:
    dt_total = dt_total.drop(columns=one_value_cols)
    print(f"Removed columns with one unique value: {one_value_cols}")

<!-- Badge 6 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#d1fae5; color:#065f46; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #a7f3d0;">
     Add Null Count Feature
    </span>
  </div>
</div>

In [11]:
# Add Null Count Feature
if dt_total.isnull().sum().sum() > 0:
    dt_total['FilasNulas'] = dt_total.isnull().sum(axis=1)

<!-- Badge 6 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#d1fae5; color:#065f46; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #a7f3d0;">
      Convert potential numeric columns to numeric type
    </span>
  </div>
</div>

In [12]:
# Convert potential numeric columns to numeric type
for col in dt_total.columns:
    if col not in ['id', 'fuente']:
        try:
            dt_total[col] = pd.to_numeric(dt_total[col], errors='coerce')
        except:
            pass

<!-- Badge 7 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#fde68a; color:#92400e; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fcd34d;">
      Encode Categorical Columns
    </span>
  </div>
</div>

In [13]:
# Encode Categorical Columns
cat_cols = dt_total.select_dtypes(include=['object']).columns.tolist()
cat_cols = [col for col in cat_cols if col != 'fuente']  # Exclude 'fuente' from encoding
for col in cat_cols:
    dt_total[col] = LabelEncoder().fit_transform(dt_total[col].astype(str))

<!-- Badge 8 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#c7d2fe; color:#3730a3; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #a5b4fc;">
      Remove Low-Variance Features
    </span>
  </div>
</div>

In [14]:
# Remove Low-Variance Features
cols_total = dt_total.columns[3:]
i_cols_borradas = 0
for c in cols_total:
    mode_val = calc_mode(dt_total[c])
    prc_repetido = (dt_total[c] == mode_val).mean()
    if prc_repetido >= 0.95:
        print(f"Removing {c} with repeated value proportion {prc_repetido:.3f}")
        dt_total = dt_total.drop(columns=c)
        i_cols_borradas += 1
print(f"Total columns removed: {i_cols_borradas}")

Total columns removed: 0


<!-- Badge 9 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#fcd5ce; color:#9d174d; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fca5a5;">
      Frequency Encoding
    </span>
  </div>
</div>

In [15]:
# Frequency Encoding
n_categories = dt_total[cols_total].nunique()
cat_features = n_categories[n_categories <= 15].index.tolist()
for c in cat_features:
    freq = dt_total[c].value_counts().to_dict()
    dt_total[f"{c}_FreqEnc"] = dt_total[c].map(freq)

<!-- Badge 10 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#d1fae5; color:#064e3b; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #86efac;">
      Handle Missing Values
    </span>
  </div>
</div>

In [16]:
# Handle Missing Values
null_cols = dt_total.columns[dt_total.isnull().sum() > 0]
for c in null_cols:
    if dt_total[c].notna().sum() > 0:  # If column has at least one non-NaN value
        median_val = dt_total[c].median()
        dt_total[c].fillna(median_val, inplace=True)
    else:  # If column is entirely NaN
        print(f"Column {c} is entirely NaN, imputing with 0")
        dt_total[c].fillna(0, inplace=True)

<!-- Badge 11 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#fde68a; color:#78350f; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fcd34d;">
      Split Train and Test
    </span>
  </div>
</div>

In [17]:
# Split Train and Test
dt = dt_total[dt_total['fuente'] == 'train'].drop(columns='fuente')
dtest = dt_total[dt_total['fuente'] == 'test'].drop(columns='fuente')

<!-- Badge 12 -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Data Preprocessing</h3></b>
  </div>
  <div style="display:flex; justify-content:center; margin-top:8px;">
    <span style="background:#c7d2fe; color:#1e3a8a; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #a5b4fc;">
      Preprocessing Pipeline
    </span>
  </div>
</div>

In [18]:
# Preprocessing Pipeline
numeric_cols = dt.select_dtypes(include=[np.number]).columns.drop(['id'], errors='ignore')
numeric_cols = [col for col in numeric_cols if dt[col].notna().sum() > 0]
if len(numeric_cols) > 0:
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_cols)
        ],
        remainder='passthrough'
    )
    dt_processed = preprocessor.fit_transform(dt)
    dtest_processed = preprocessor.transform(dtest)
    all_cols = numeric_cols + [col for col in dt.columns if col not in numeric_cols]
    dt = pd.DataFrame(dt_processed, columns=all_cols)
    dtest = pd.DataFrame(dtest_processed, columns=all_cols)
else:
    print("No valid numeric columns to scale. Using all columns as-is.")
    dt = dt.copy()
    dtest = dtest.copy()
    if dt.drop(columns=['id'], errors='ignore').shape[1] == 0:
        raise ValueError("No features available for modeling after preprocessing.")
dt['id'] = dt['id'].astype(int)
dtest['id'] = dtest['id'].astype(int)

<!-- ================= H2 HEADER ================= -->
<div style="border-radius:25px 70px; padding:14px 18px; margin:0 0 18px 0;
            font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">
    <b><h2 style="margin:0; color:#ffffff;">Model Training and Prediction</h2></b>
  </div>
</div>

<!-- ================= BADGE ================= -->
<div style="display:flex; justify-content:center; margin:12px 0; font-family:Inter, system-ui, 'Segoe UI', Roboto, Arial, sans-serif;">
  <span style="background:#dcfce7; color:#14532d; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bbf7d0;">
    Initialize OOF Predictions & Submission DataFrames
  </span>
</div>


In [19]:
# Setup
oof = pd.DataFrame({'target': Y})
sub_seeds = sub.copy()
sub_seeds.iloc[:, 1:] = 0
scores = []
SEEDS = [1975, 2000, 2503, 1511, 2604]

<!-- ================= H3 HEADER ================= -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">LightGBM</h3></b>
  </div>
  <div style="display:flex; gap:10px; justify-content:center; margin-top:8px; flex-wrap:wrap;">
    <span style="background:#dbeafe; color:#1e3a8a; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bfdbfe;">
      Predictions
    </span>
    <span style="background:#dcfce7; color:#14532d; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bbf7d0;">
      Test Predictions
    </span>
    <span style="background:#fde68a; color:#92400e; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fcd34d;">
      Feature Importance
    </span>
  </div>
</div>


In [20]:
for s in SEEDS:
    print(f"\nSeed: {s}")
    np.random.seed(s)
    name_seed = f"Seed_{s}"
    sub_seeds[name_seed] = 0
    if dt.shape[0] == 0:
        raise ValueError("Training dataset (dt) is empty. Check data loading or preprocessing steps.")
    kf = KFold(n_splits=10, shuffle=True, random_state=s)
    pred_lgbm_total = np.zeros(len(dtest))
    sub_temp = pd.DataFrame({'id': sub['id']})
    auc_lgbm_total = 0

    for i_fold, (train_idx, val_idx) in enumerate(kf.split(dt), 1):
        print(f"\nLGBM Fold {i_fold} - 10")
        X_train, y_train = dt.iloc[train_idx].drop(columns='id'), Y[train_idx]
        X_val, y_val = dt.iloc[val_idx].drop(columns='id'), Y[val_idx]

        # LightGBM model
        lgb_params = {
            'random_state': 0,
            'n_estimators': 4500,
            'learning_rate': 0.005,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse'
        }

        model = LGBMRegressor(**lgb_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_metric='rmse',
            callbacks=[early_stopping(stopping_rounds=500), log_evaluation(period=250)]
        )

        # Predictions
        test_lgbm = model.predict(X_val)
        oof.loc[val_idx, f"LGBM_Seed_{s}"] = test_lgbm
        auc_lgbm = rmse(y_val, test_lgbm)
        auc_lgbm_total += auc_lgbm
        scores.append(auc_lgbm)
        print(f"LGBM Fold {i_fold} RMSE: {auc_lgbm:.4f}, Avg RMSE: {np.mean(scores):.4f}")

        # Test predictions
        pred_lgbm = model.predict(dtest.drop(columns='id'))
        pred_lgbm_total += pred_lgbm
        sub_temp[f"Seed_s{s}_Fold_{i_fold}"] = pred_lgbm
        sub_seeds[name_seed] += pred_lgbm / 10

        # Feature importance
        importance = pd.DataFrame({
            'Feature': X_train.columns,
            'Importance': model.feature_importances_
        })
        importance.to_csv("ImportanciaLGBM_v1.csv", index=False)
        free()

    pred_lgbm_total /= 10
    print(f"AUC LGBM Seed {s}: {np.mean(scores):.4f}")


Seed: 1975

LGBM Fold 1 - 10
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022742 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 471747, number of used features: 9
[LightGBM] [Info] Start training from score 119.035388
Training until validation scores don't improve for 500 rounds
[250]	training's rmse: 26.4407	valid_1's rmse: 26.4797
[500]	training's rmse: 26.4241	valid_1's rmse: 26.4799
[750]	training's rmse: 26.4087	valid_1's rmse: 26.4809
Early stopping, best iteration is:
[296]	training's rmse: 26.4377	valid_1's rmse: 26.4795
LGBM Fold 1 RMSE: 26.4795, Avg RMSE: 26.4795

LGBM Fold 2 - 10
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030685 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the tra

<!-- ================= H3 HEADER ================= -->
<div style="border-radius:15px; padding:10px 14px; margin:10px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:130%; margin:0; padding:0;">
    <b><h3 style="margin:0; color:#ffffff;">Save Outputs (Saves Predictions)</h3></b>
  </div>
  <div style="display:flex; gap:10px; justify-content:center; margin-top:8px; flex-wrap:wrap;">
    <span style="background:#dbeafe; color:#1e3a8a; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bfdbfe;">
      Mean
    </span>
    <span style="background:#dcfce7; color:#14532d; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bbf7d0;">
      Median
    </span>
    <span style="background:#fde68a; color:#92400e; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fcd34d;">
      Fold-wise
    </span>
    <span style="background:#fcd5ce; color:#881337; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #fda4af;">
      OOF
    </span>
    <span style="background:#e9d5ff; color:#581c87; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #d8b4fe;">
      Seed-wise
    </span>
  </div>
</div>


In [21]:
# Save Outputs
sub.iloc[:, 1] = sub_temp.iloc[:, 1:].mean(axis=1)
sub.to_csv("subLGBMFit_v1.csv", index=False)

sub.iloc[:, 1] = sub_temp.iloc[:, 1:].median(axis=1)
sub.to_csv("subLGBMFitMedian_v1.csv", index=False)

sub_temp.to_csv("subFoldLGBMFit_v1.csv", index=False)
oof.to_csv("OOF_LGBMFit_v1.csv", index=False)
sub_seeds.to_csv("subSeedsLGBM_v1.csv", index=False)

<!-- ================= H2 HEADER ================= -->
<div style="border-radius:25px 70px; padding:14px 18px; margin:0 0 18px 0;
            font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">
    <b><h2 style="margin:0; color:#ffffff;">Evaluate Performance</h2></b>
  </div>
</div>

<!-- ================= BADGE ================= -->
<div style="display:flex; justify-content:center; margin:12px 0; font-family:Inter, system-ui, 'Segoe UI', Roboto, Arial, sans-serif;">
  <span style="background:#dcfce7; color:#14532d; padding:6px 14px; border-radius:999px; font-size:13px; border:1px solid #bbf7d0;">
    Print RMSE per seed & mean RMSE
  </span>
</div>


In [22]:
# Evaluate Performance
for s in SEEDS:
    print(f"RMSE Seed {s}: {rmse(oof['target'], oof[f'LGBM_Seed_{s}']):.4f}")

auc_mean = np.mean([rmse(oof['target'], oof[f'LGBM_Seed_{s}']) for s in SEEDS])
print(f"Mean RMSE: {auc_mean:.4f}")

RMSE Seed 1975: 26.4583
RMSE Seed 2000: 26.4578
RMSE Seed 2503: 26.4581
RMSE Seed 1511: 26.4584
RMSE Seed 2604: 26.4585
Mean RMSE: 26.4582


<!-- ================= HEADER + BADGES MERGED ================= -->
<div style="border-radius:25px 70px; padding:14px 18px; margin:0 0 18px 0; font-family:'Quicksand', sans-serif; background:#24a695; overflow:hidden;">
  <div style="text-align:center; color:#ffffff; font-size:170%; margin:0; padding:0;">
    <b><h3 style="text-align:center; color:#ffffff; font-size:150%; margin:0; padding:0;">Thank You</h3></b>
  </div>
</div>

<div style="display:flex; gap:8px; flex-wrap:wrap; margin-top:10px; font-family:Inter, system-ui, 'Segoe UI', Roboto, Arial, sans-serif;">
  <a href="#Table-of-contents" style="margin-left:auto; font-size:12px; color:red; text-decoration:none;">Back to top ↑</a>
</div>
